# Import libraries and modules

In [1]:
# selenium import here

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [2]:
# Other import here

import pandas as pd
import time


# Start and Setup

In [3]:

start_time = time.time()
url = "https://www.ntu.ac.uk/search?query=Faculty%20member%20of%20biology&category=&sort=relevance&category=Staff"
s = Service("chromedriver-win32/chromedriver")

driver = webdriver.Chrome(service = s)
driver.get(url)
time.sleep(3)


In [4]:
# Close cookie banner if present

try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "accept-cookies"))).click()
except:
    pass

# Filter of staff

In [5]:
# select staff checkbox members

# driver.maximize_window()
# sttaf_chechbox = driver.find_element(By.XPATH, value = '//label[@for = "Staff"]')
# sttaf_chechbox.click()
# time.sleep(5)

### Load all results of search

In [6]:

result_count = driver.find_element(By.XPATH, value= '//strong[@class = "resultCount"]').text
result_count = int(result_count)
print(result_count)


160


In [7]:

if result_count % 10 == 0:
    page_count = result_count // 10 - 1
else:
    page_count = result_count // 10

for i in range(page_count):
        next_page = driver.find_element(By.XPATH, value = '//a[@class = "more-results" ]')
        next_page.get_attribute("data-more")
        next_page.click()
        time.sleep(10)
    

### get list of sttuf link 

In [22]:

url_list = []

all_items_url = driver.find_elements(By.XPATH, value = '//li[@class = "result clearfix"]')

for i in range(len(all_items_url)):
# get link of profile for each prof (item) and go that site

    item = driver.find_elements(By.XPATH, value = './/li[@class = "result clearfix"]')[i]
    links_prof_page = item.find_element(By.XPATH , value = './/h2[@class = "result-title  skim-bottom"]//a')
    links_prof_page = links_prof_page.get_attribute('href')
    url_list.append(links_prof_page)


In [10]:
# store urls for each member

url_df = pd.DataFrame({'url': url_list})
url_df.to_csv("url_df.csv", index=False)

### click to each link and extract all data

In [13]:

data = []

for url in url_list:
    try:
        # In each page of profile,we get name, email, department and research area
        driver.get(url)
        page_prof = driver.find_elements(By.XPATH , value = '//main[@class = "main"]')
        
        for section in page_prof:
            
                    try:
                        name = section.find_element(By.XPATH , value = ".//h1[@class = 'page-heading skim-top skim-bottom']").text
                    except:
                        name = "Unknown"
                        
                    try:
                        email = section.find_element(By.XPATH  , value = './/a[@itemprop= "email"]').text
                    except:
                        email = "Unknown"

                    try:
                        department = section.find_element(By.XPATH, value= './/h3[@class = "skim-top skim-bottom"]').text
                    except:
                        department = 'Unknown' 

                    try:
                        research_area = section.find_element(By.XPATH , value = './/div[@class= "grey-shade key-panel"]').text
                    except:
                        research_area = "Unknown"

                    data.append([name, email, department, research_area])
        
    
    except:
        print("finished!")
        break

print('How long does it take to scrape the whole webpage? {} seconds'.format(time.time() - start_time))


finished!
How long does it take to scrape the whole webpage? 10752.698683738708 seconds


In [14]:
len(data)

63

# Create Pandas Dataframe

In [15]:

df = pd.DataFrame(data, columns = ['Name', 'email', 'department' , 'research_area'])
df


,Name,email,department,research_area
0,Richard McFarland,richard.mcfarland@ntu.ac.uk,School of Social Sciences,Research areas\nThe underlying objective of my...
1,Adnan Saithna,adnan.saithna@ntu.ac.uk,,Research areas\nDr Saithna has active clinical...
2,Sergio Rutella,sergio.rutella@ntu.ac.uk,School of Science & Technology,Research areas\nAreas of clinical research int...
3,petros siegkas,petros.siegkas@ntu.ac.uk,School of Science & Technology,Research areas\nDr Siegkas’ research within th...
4,Dr Seema Patel,seema.patel02@ntu.ac.uk,Nottingham Law School,"Research areas\nSeema has a PhD in sports law,..."
...,...,...,...,...
58,Richard Yarnell,richard.yarnell@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...
59,Mufti Mahmud,mufti.mahmud@ntu.ac.uk,Department of Computer Science; Computing and ...,Research areas\nPostdoctoral researchers\nAndr...
60,Aldo Gutierrez,aldo.gutierrez@ntu.ac.uk,School of Science & Technology,Research areas\nDr Gutierrez is a member of th...
61,Louise Gentle,louise.gentle@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...


In [17]:

url_df = df.drop_duplicates()
df = df.drop_duplicates()


In [18]:
len(url_df)

63

In [19]:
len(df)

63

In [20]:

result = pd.concat([df, url_df], axis=1)
result


,Name,email,department,research_area,Name,email,department,research_area
0,Richard McFarland,richard.mcfarland@ntu.ac.uk,School of Social Sciences,Research areas\nThe underlying objective of my...,Richard McFarland,richard.mcfarland@ntu.ac.uk,School of Social Sciences,Research areas\nThe underlying objective of my...
1,Adnan Saithna,adnan.saithna@ntu.ac.uk,,Research areas\nDr Saithna has active clinical...,Adnan Saithna,adnan.saithna@ntu.ac.uk,,Research areas\nDr Saithna has active clinical...
2,Sergio Rutella,sergio.rutella@ntu.ac.uk,School of Science & Technology,Research areas\nAreas of clinical research int...,Sergio Rutella,sergio.rutella@ntu.ac.uk,School of Science & Technology,Research areas\nAreas of clinical research int...
3,petros siegkas,petros.siegkas@ntu.ac.uk,School of Science & Technology,Research areas\nDr Siegkas’ research within th...,petros siegkas,petros.siegkas@ntu.ac.uk,School of Science & Technology,Research areas\nDr Siegkas’ research within th...
4,Dr Seema Patel,seema.patel02@ntu.ac.uk,Nottingham Law School,"Research areas\nSeema has a PhD in sports law,...",Dr Seema Patel,seema.patel02@ntu.ac.uk,Nottingham Law School,"Research areas\nSeema has a PhD in sports law,..."
...,...,...,...,...,...,...,...,...
58,Richard Yarnell,richard.yarnell@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...,Richard Yarnell,richard.yarnell@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...
59,Mufti Mahmud,mufti.mahmud@ntu.ac.uk,Department of Computer Science; Computing and ...,Research areas\nPostdoctoral researchers\nAndr...,Mufti Mahmud,mufti.mahmud@ntu.ac.uk,Department of Computer Science; Computing and ...,Research areas\nPostdoctoral researchers\nAndr...
60,Aldo Gutierrez,aldo.gutierrez@ntu.ac.uk,School of Science & Technology,Research areas\nDr Gutierrez is a member of th...,Aldo Gutierrez,aldo.gutierrez@ntu.ac.uk,School of Science & Technology,Research areas\nDr Gutierrez is a member of th...
61,Louise Gentle,louise.gentle@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...,Louise Gentle,louise.gentle@ntu.ac.uk,School of Animal Rural & Environmental Sciences,Research areas\nAreas of research interest inc...


In [21]:
result.to_csv("faculty_member_notthingham.csv", index=False)